In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand, row_number
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
import random


spark = SparkSession.builder \
    .appName("EuroCup2024") \
    .getOrCreate()

# Team names
european_teams = [
    ("Czech", "A"), ("Croatia", "A"), ("Hungary", "A"), ("Serbia", "A"),
    ("England", "B"), ("Scotland", "B"), ("Italy", "B"), ("Greece", "B"),
    ("Slovenia", "C"), ("Denmark", "C"), ("Switzerland", "C"), ("Spain", "C"), 
    ("Finland", "D"), ("Nether lands", "D"), ("Austria", "D"),("France", "D"),
    ("Belgium", "E"),("Slovakia", "E"), ("Romania", "E"),("Iceland", "E"),
    ("Türkiye", "F"), ("Albania", "F"), ("Portugal", "F"), ("Germany", "F")
]

group_stage_df = spark.createDataFrame(european_teams, ["teamname", "group"])


print("Table 1: Grouping")
group_stage_df.show()


Table 1: Group Stage
+------------+-----+
|    teamname|group|
+------------+-----+
|       Czech|    A|
|     Croatia|    A|
|     Hungary|    A|
|      Serbia|    A|
|     England|    B|
|    Scotland|    B|
|       Italy|    B|
|      Greece|    B|
|    Slovenia|    C|
|     Denmark|    C|
| Switzerland|    C|
|       Spain|    C|
|     Finland|    D|
|Nether lands|    D|
|     Austria|    D|
|      France|    D|
|     Belgium|    E|
|    Slovakia|    E|
|     Romania|    E|
|     Iceland|    E|
+------------+-----+
only showing top 20 rows



In [0]:
#Knock out Phase according to rules
values = [
    ('ABCD', 'A', 'D', 'B', 'C'),
    ( 'ABCE', 'A', 'E', 'B', 'C'),
    ( 'ABCF', 'A', 'F', 'B', 'C'),
    ( 'ABDE', 'D', 'E', 'A', 'B'),
    ( 'ABDF', 'D', 'F', 'A', 'B'),
    ( 'ABEF', 'E', 'F', 'B', 'A'),
    ( 'ACDE', 'E', 'D', 'C', 'A'),
    ( 'ACDF', 'F', 'D', 'C', 'A'),
    ( 'ACEF', 'E', 'F', 'C', 'A'),
    ( 'ADEF', 'E', 'F', 'D', 'A'),
    ( 'BCDE', 'E', 'D', 'B', 'C'),
    ( 'BCDF', 'F', 'D', 'C', 'B'),
    ( 'BCEF', 'F', 'E', 'C', 'B'),
    ( 'BDEF', 'F', 'E', 'D', 'B'),
    ( 'CDEF', 'F', 'E', 'D', 'C')
]

df_knockout_phase = spark.createDataFrame(values, ["Third_placed_qualified_teams", "B", "C", "E", "F"])
df_knockout_phase.show()



+----------------------------+---+---+---+---+
|Third_placed_qualified_teams|  B|  C|  E|  F|
+----------------------------+---+---+---+---+
|                        ABCD|  A|  D|  B|  C|
|                        ABCE|  A|  E|  B|  C|
|                        ABCF|  A|  F|  B|  C|
|                        ABDE|  D|  E|  A|  B|
|                        ABDF|  D|  F|  A|  B|
|                        ABEF|  E|  F|  B|  A|
|                        ACDE|  E|  D|  C|  A|
|                        ACDF|  F|  D|  C|  A|
|                        ACEF|  E|  F|  C|  A|
|                        ADEF|  E|  F|  D|  A|
|                        BCDE|  E|  D|  B|  C|
|                        BCDF|  F|  D|  C|  B|
|                        BCEF|  F|  E|  C|  B|
|                        BDEF|  F|  E|  D|  B|
|                        CDEF|  F|  E|  D|  C|
+----------------------------+---+---+---+---+



In [0]:
#DataFrame as temporary view
group_stage_df.createOrReplaceTempView("group_of_24")

# Executing the SQL query
group_of_16 = spark.sql("""
WITH cte4 AS (
WITH cte3 AS (
WITH cte2 AS (
WITH cte1 AS (
WITH cte AS (
SELECT
  a.teamname AS team1,
  a.group AS group1,
  1 AS mp ,
  b.teamname AS team2,
  b.group AS group2,
  FLOOR(RAND() * 5) AS goals1,
  FLOOR(RAND() * 5) AS goals2,
  CASE 
    WHEN goals1 > goals2 THEN 3
    WHEN goals1 == goals2 THEN 1
    ELSE 0
    END as point1,
  CASE
    WHEN goals2 > goals1 THEN 3
    WHEN goals2 == goals1 THEN 1
    ELSE 0
    END as point2 ,
  CASE
    WHEN point1 == 3 THEN 1
    ELSE 0
    END as w1,
  CASE
    WHEN point2 == 3 THEN 1
    ELSE 0
    END as w2, 
 CASE
    WHEN point1 == 1 THEN 1
    ELSE 0
    END as d1,
  CASE
    WHEN point2 == 1 THEN 1
    ELSE 0
    END as d2,  
  CASE
    WHEN point1 == 0 THEN 1
    ELSE 0
    END as l1,
  CASE
    WHEN point2 == 0 THEN 1
    ELSE 0
    END as l2
FROM
  group_of_24 a
CROSS JOIN
  group_of_24 b
ON
  a.group = b.group AND a.teamname < b.teamname
)
select team1 as team,group1 as groups ,mp,goals1 as gf, goals2 as ga,point1 as pts ,w1 as w,d1 as d,l1 as l from cte 
union all
select team2 as team,group2 as groups,mp,goals2 as gf, goals1 as ga,point2 as pts ,w2 as w,d2 as d,l2 as l from cte 
)
select team, groups , sum(w) as w, sum(d) as d, sum(l) as l, sum(mp) as mp,sum(gf) as gf,sum(ga) as ga,sum(gf -ga) as gd,sum(pts) as pts 
from cte1 
group by team,groups
order by groups, pts desc, gd desc, gf desc
)
select *,
row_number() over(partition by groups order by pts desc, gd desc, gf desc) as rank
from cte2
)

select * 
from cte3
where rank <= 3
order by rank,pts desc,gd desc, gf desc
limit 16
)
select * from cte4""")

# Assume df is your DataFrame
group_of_16 = group_of_16.withColumnRenamed("team", "Team") \
       .withColumnRenamed("groups", "Groups") \
       .withColumnRenamed("w", "Wins") \
       .withColumnRenamed("d", "Draws") \
       .withColumnRenamed("l", "Losses") \
       .withColumnRenamed("mp", "MatchesPlayed") \
       .withColumnRenamed("gf", "GoalsFor") \
       .withColumnRenamed("ga", "GoalsAgainst") \
       .withColumnRenamed("gd", "GoalDifference") \
       .withColumnRenamed("pts", "Points")

group_of_16.display()

Team,Groups,Wins,Draws,Losses,MatchesPlayed,GoalsFor,GoalsAgainst,GoalDifference,Points,rank
Scotland,B,3,0,0,3,9,1,8,9,1
Romania,E,3,0,0,3,9,3,6,9,1
France,D,2,1,0,3,8,6,2,7,1
Hungary,A,2,0,1,3,8,4,4,6,1
Germany,F,2,0,1,3,7,4,3,6,1
Spain,C,1,2,0,3,7,3,4,5,1
Serbia,A,2,0,1,3,8,5,3,6,2
Finland,D,2,0,1,3,8,5,3,6,2
Portugal,F,2,0,1,3,6,4,2,6,2
England,B,2,0,1,3,4,3,1,6,2


In [0]:
group_of_16.createOrReplaceTempView("group_of_16")
df_knockout_phase.createOrReplaceTempView("df_knockout_phase")

In [0]:
%sql
WITH combo AS (
WITH group_of_16_ordered AS (
  SELECT * FROM group_of_16
  WHERE rank = 3
  ORDER BY groups
  limit 4
)
SELECT  concat_ws('',array_agg(groups) )AS pattern
FROM group_of_16_ordered
)
select * from
combo
left join df_knockout_phase
on pattern = Third_placed_qualified_teams

pattern,Third_placed_qualified_teams,B,C,E,F
ABCF,ABCF,A,F,B,C


In [0]:
%sql
CREATE or replace table group_of_16_brackets as(
WITH combination AS (
WITH pattern AS (
WITH group_of_16_ordered AS (
  SELECT * FROM group_of_16
  WHERE rank = 3
  ORDER BY groups
  limit 4
)
SELECT  concat_ws('',array_agg( groups) )AS pattern
FROM group_of_16_ordered 
)
select * from
pattern
left join df_knockout_phase
on pattern = Third_placed_qualified_teams
)
select team, groups,rank, 1 as row_num from group_of_16 where rank = 1 and groups ='B'
union all
select team, groups,rank , 2 as row_num from group_of_16 where rank = 3 and groups =  (select B from combination)
union all
select team,groups,rank, 3 as row_num from group_of_16 where rank = 1 and groups ='A'
union all
select team,groups,rank, 4 as row_num from group_of_16 where rank = 2 and groups ='C'
union all
select team,groups,rank, 5 as row_num from group_of_16 where rank = 1 and groups ='F'
union all
select team,groups,rank, 6 as row_num from group_of_16 where rank = 3 and groups =(select E from combination)
union all
select team,groups,rank, 7 as row_num from group_of_16 where rank = 2 and groups ='D'
union all
select team,groups,rank, 8 as row_num from group_of_16 where rank = 2 and groups ='E'
union all
select team,groups,rank, 9 as row_num from group_of_16 where rank = 1 and groups ='E'
union all
select team,groups,rank, 10 as row_num from group_of_16 where rank = 3 and groups =(select F from combination)
union all
select team,groups,rank, 11 as row_num from group_of_16 where rank = 1 and groups ='D'
union all
select team,groups,rank, 12 as row_num from group_of_16 where rank = 2 and groups ='F'
union all
select team,groups,rank, 13 as row_num from group_of_16 where rank = 1 and groups ='C'
union  all
select team,groups,rank, 14 as row_num from group_of_16 where rank = 3 and groups =(select C from combination)
union all
select team,groups,rank, 15 as row_num from group_of_16 where rank = 2 and groups ='A'
union all
select team,groups,rank, 16 as row_num from group_of_16 where rank = 2 and groups ='B'
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from group_of_16_brackets
order by row_num

team,groups,rank,row_num
Scotland,B,1,1
Czech,A,3,2
Hungary,A,1,3
Switzerland,C,2,4
Germany,F,1,5
Greece,B,3,6
Finland,D,2,7
Slovakia,E,2,8
Romania,E,1,9
Denmark,C,3,10


In [0]:
%sql
WITH group_of_16_ordered AS (
  SELECT * FROM group_of_16
  WHERE rank = 3
  ORDER BY groups
  limit 4
) 
select * FROM group_of_16_ordered

Team,Groups,Wins,Draws,Losses,MatchesPlayed,GoalsFor,GoalsAgainst,GoalDifference,Points,rank
Czech,A,1,1,1,3,6,9,-3,4,3
Greece,B,1,0,2,3,6,8,-2,3,3
Denmark,C,1,1,1,3,4,6,-2,4,3
Türkiye,F,2,0,1,3,7,7,0,6,3


In [0]:
%sql
WITH combination AS (
WITH pattern AS (
WITH group_of_16_ordered AS (
  SELECT * FROM group_of_16
  WHERE rank = 3
  ORDER BY groups
  limit 4
)
SELECT  concat_ws('',array_agg( groups) )AS pattern
FROM group_of_16_ordered 
)
select * from
pattern
left join df_knockout_phase
on pattern = Third_placed_qualified_teams)
select * from combination

pattern,Third_placed_qualified_teams,B,C,E,F
ABCF,ABCF,A,F,B,C


In [0]:
group16 = spark.sql("""SELECT
    t1.team AS team1,
    t1.groups AS group1,
    t1.rank AS rank1,
    t1.row_num AS row1,
    t2.team AS team2,
    t2.groups AS group2,
    t2.rank AS rank2,
    t2.row_num AS row2,
    CASE WHEN FLOOR(RAND() * 10) > 5 then team1 else team2 end as won
FROM
    group_of_16_brackets t1
JOIN
    group_of_16_brackets t2
ON
    t1.row_num % 2 = 1 AND t2.row_num % 2 = 0 AND t1.row_num + 1 = t2.row_num
""")

group16.show()

+--------+------+-----+----+-----------+------+-----+----+-----------+
|   team1|group1|rank1|row1|      team2|group2|rank2|row2|        won|
+--------+------+-----+----+-----------+------+-----+----+-----------+
|Scotland|     B|    1|   1|      Czech|     A|    3|   2|      Czech|
| Finland|     D|    2|   7|   Slovakia|     E|    2|   8|   Slovakia|
| Germany|     F|    1|   5|     Greece|     B|    3|   6|    Germany|
| Romania|     E|    1|   9|    Denmark|     C|    3|  10|    Romania|
| Hungary|     A|    1|   3|Switzerland|     C|    2|   4|Switzerland|
|  France|     D|    1|  11|   Portugal|     F|    2|  12|     France|
|  Serbia|     A|    2|  15|    England|     B|    2|  16|     Serbia|
|   Spain|     C|    1|  13|    Türkiye|     F|    3|  14|      Spain|
+--------+------+-----+----+-----------+------+-----+----+-----------+



In [0]:
group16_winner = group16.select("won")
group16_winner.show()

window_spec = Window.orderBy(F.monotonically_increasing_id())
group16_winner = group16_winner.withColumn("teamId", F.row_number().over(window_spec))

# Create a DataFrame with odd teamId teams as team1 and even teamId teams as team2
team1_df = group16_winner.filter((F.col("teamId") % 2) != 0).withColumnRenamed("teamId", "team1_id").withColumnRenamed("won", "team1")
team1_df = team1_df.withColumn("teamId", F.row_number().over(window_spec)).drop('team1_id')
team2_df = group16_winner.filter((F.col("teamId") % 2) == 0).withColumnRenamed("teamId", "team2_id").withColumnRenamed("won", "team2")
team2_df = team2_df.withColumn("teamId", F.row_number().over(window_spec)).drop('team2_id')
team1_df.show()
team2_df.show()

# Join team1 and team2 DataFrames to get the final DataFrame
quarterfinals_df = team1_df.join(team2_df, ["teamId"], "inner")
# Add a column for winner (randomly chosen between team1 and team2)
quarterfinals_df = quarterfinals_df.withColumn("winner", F.when(F.rand() > 0.5, quarterfinals_df["team1"]).otherwise(quarterfinals_df["team2"]))

# Select necessary columns
quarterfinals_df = quarterfinals_df.select("team1", "team2", "winner")

quarterfinals_df.show()

+-----------+
|        won|
+-----------+
|      Czech|
|   Slovakia|
|    Germany|
|    Romania|
|Switzerland|
|     France|
|     Serbia|
|      Spain|
+-----------+

+-----------+------+
|      team1|teamId|
+-----------+------+
|      Czech|     1|
|    Germany|     2|
|Switzerland|     3|
|     Serbia|     4|
+-----------+------+

+--------+------+
|   team2|teamId|
+--------+------+
|Slovakia|     1|
| Romania|     2|
|  France|     3|
|   Spain|     4|
+--------+------+

+-----------+--------+-------+
|      team1|   team2| winner|
+-----------+--------+-------+
|      Czech|Slovakia|  Czech|
|    Germany| Romania|Germany|
|Switzerland|  France| France|
|     Serbia|   Spain| Serbia|
+-----------+--------+-------+



In [0]:
quarterfinals_df = quarterfinals_df.select("winner")
quarterfinals_df.show()

window_spec = Window.orderBy(F.monotonically_increasing_id())
quarterfinals_df = quarterfinals_df.withColumn("teamId", F.row_number().over(window_spec))

# Create a DataFrame with odd teamId teams as team1 and even teamId teams as team2
team1_df = quarterfinals_df.filter((F.col("teamId") % 2) != 0).withColumnRenamed("teamId", "team1_id").withColumnRenamed("winner", "team1")
team1_df = team1_df.withColumn("teamId", F.row_number().over(window_spec)).drop('team1_id')
team2_df = quarterfinals_df.filter((F.col("teamId") % 2) == 0).withColumnRenamed("teamId", "team2_id").withColumnRenamed("winner", "team2")
team2_df = team2_df.withColumn("teamId", F.row_number().over(window_spec)).drop('team2_id')
team1_df.show()
team2_df.show()

# Join team1 and team2 DataFrames to get the final DataFrame
semifinals_df = team1_df.join(team2_df, ["teamId"], "inner")
# Add a column for winner (randomly chosen between team1 and team2)
semifinals_df = semifinals_df.withColumn("winner", F.when(F.rand() > 0.5, semifinals_df["team1"]).otherwise(semifinals_df["team2"]))

# Select necessary columns
semifinals_df = semifinals_df.select("team1", "team2", "winner")

semifinals_df.show()

+-------+
| winner|
+-------+
|  Czech|
|Germany|
| France|
| Serbia|
+-------+

+------+------+
| team1|teamId|
+------+------+
| Czech|     1|
|France|     2|
+------+------+

+-------+------+
|  team2|teamId|
+-------+------+
|Germany|     1|
| Serbia|     2|
+-------+------+

+------+-------+------+
| team1|  team2|winner|
+------+-------+------+
| Czech|Germany| Czech|
|France| Serbia|France|
+------+-------+------+



In [0]:
semifinals_df = semifinals_df.withColumn("random_number", F.rand())
winner = semifinals_df.select(F.when(F.col("random_number") > 0.5, semifinals_df["winner"]).otherwise(semifinals_df["winner"])).first()[0]

print("Winner:", winner)

Winner: Czech
